<a href="https://colab.research.google.com/github/LeticiaGiVi/prototipoGoodWe/blob/main/protipo_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import base64
import requests
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta, time as dtime
from typing import Literal, Dict, Any, List, Optional

In [ ]:
Region = Literal["us", "eu"]

class SEMSClient:
    BASE_URLS = {
        "us": "https://us.semsportal.com",
        "eu": "https://eu.semsportal.com",
    }

    def __init__(self, account: str, password: str, login_region: Region = "us", data_region: Region = "eu"):
        self.account = account
        self.password = password
        self.login_region = login_region
        self.data_region = data_region
        self._token = None

    def _initial_token(self) -> str:
        """Gera o Token inicial (pré-login)"""
        original = {
            "uid": "",
            "timestamp": 0,
            "token": "",
            "client": "web",
            "version": "",
            "language": "en"
        }
        b = json.dumps(original).encode("utf-8")
        return base64.b64encode(b).decode("utf-8")

    def login(self) -> str:
        """Faz o crosslogin e armazena o token"""
        url = f"{self.BASE_URLS[self.login_region]}/api/v2/common/crosslogin"
        headers = {
            "Token": self._initial_token(),
            "Content-Type": "application/json",
            "Accept": "*/*"
        }
        payload = {
            "account": self.account,
            "pwd": self.password,
            "agreement_agreement": 0,
            "is_local": False
        }

        try:
            response = requests.post(url, json=payload, headers=headers, timeout=30)
            response.raise_for_status()

            js = response.json()
            if "data" not in js or js.get("code") not in (0, 1, 200):
                raise RuntimeError(f"Login falhou: {js}")

            data_to_string = json.dumps(js["data"])
            self._token = base64.b64encode(data_to_string.encode("utf-8")).decode("utf-8")
            return self._token

        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"Erro de conexão durante login: {e}")
        except Exception as e:
            raise RuntimeError(f"Erro durante login: {e}")

    def get_inverter_data_by_column(self, inverter_sn: str, column: str, date_str: str) -> Dict[str, Any]:
        if not self._token:
            raise RuntimeError("Token não disponível. Execute login() primeiro.")

        url = f"{self.BASE_URLS[self.data_region]}/api/PowerStationMonitor/GetInverterDataByColumn"
        headers = {
            "Token": self._token,
            "Content-Type": "application/json",
            "Accept": "*/*"
        }
        payload = {
            "date": date_str,
            "column": column,
            "id": inverter_sn
        }

        try:
            response = requests.post(url, json=payload, headers=headers, timeout=30)
            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"Erro de conexão ao buscar dados: {e}")
        except Exception as e:
            raise RuntimeError(f"Erro ao buscar dados: {e}")

class SEMSDataProcessor:
    @staticmethod
    def parse_column_timeseries(resp_json: Dict[str, Any], column_name: str) -> pd.DataFrame:

        def _parse_time(ts):
            v = pd.to_datetime(ts, errors='coerce')
            if pd.isna(v):
                try:
                    v = pd.to_datetime(ts, dayfirst=True, errors='coerce')
                except:
                    v = pd.NaT
            return v

        items = []
        if isinstance(resp_json, dict):
            data_obj = resp_json.get('data')
            if isinstance(data_obj, dict):
                for key in ('column1', 'column2', 'column3', 'items', 'list', 'datas', 'result'):
                    if key in data_obj and isinstance(data_obj[key], list):
                        items = data_obj[key]
                        break

            if not items:
                for key in ('data', 'items', 'list', 'result', 'datas'):
                    if key in resp_json and isinstance(resp_json[key], list):
                        items = resp_json[key]
                        break

        if not items:
            return pd.DataFrame()

        times, values = [], []
        for item in items:
            if not isinstance(item, dict):
                continue

            timestamp = (item.get('time') or
                        item.get('date') or
                        item.get('collectTime') or
                        item.get('cTime') or
                        item.get('tm'))

            if column_name in item:
                value = item.get(column_name)
            else:
                value = (item.get('value') or
                        item.get('v') or
                        item.get('val') or
                        item.get('column'))

            if timestamp is None or value is None:
                continue

            parsed_time = _parse_time(timestamp)
            if pd.isna(parsed_time):
                continue

            try:
                parsed_value = float(str(value).replace(',', '.'))
            except:
                continue

            times.append(parsed_time)
            values.append(parsed_value)

        if not times:
            return pd.DataFrame()

        df = pd.DataFrame({'time': times, column_name: values})
        return df.dropna().sort_values('time').reset_index(drop=True)


def fetch_weekly_sems_data(account: str,
                          password: str,
                          inverter_sn: str,
                          start_date: date,
                          columns: List[str],
                          login_region: Region = "us",
                          data_region: Region = "eu") -> pd.DataFrame:

    print(f"Buscando dados da semana começando em {start_date}")

    client = SEMSClient(account, password, login_region, data_region)
    processor = SEMSDataProcessor()

    try:
        client.login()
        print("Login realizado com sucesso!")
    except Exception as e:
        raise RuntimeError(f"Erro no login: {e}")

    weekly_data = []

    for day_offset in range(7):
        current_date = start_date + timedelta(days=day_offset)
        date_str = datetime.combine(current_date, dtime(0, 0)).strftime("%Y-%m-%d %H:%M:%S")

        print(f"\nDia {day_offset + 1}/7: {current_date}")

        day_dataframes = []
        for column in columns:
            try:
                response = client.get_inverter_data_by_column(inverter_sn, column, date_str)
                df_column = processor.parse_column_timeseries(response, column)

                if not df_column.empty:
                    day_dataframes.append(df_column)
                    print(f"  {column}: {len(df_column)} registros")
                else:
                    print(f"  {column}: sem dados")

            except Exception as e:
                print(f"  Erro em {column}: {e}")

        if day_dataframes:
            day_df = day_dataframes[0]
            for df_next in day_dataframes[1:]:
                day_df = pd.merge_asof(
                    day_df.sort_values("time"),
                    df_next.sort_values("time"),
                    on="time",
                    direction="nearest"
                )
            weekly_data.append(day_df)

    if weekly_data:
        final_df = pd.concat(weekly_data, ignore_index=True)
        final_df = final_df.sort_values("time").reset_index(drop=True)
        print(f"\nDados semanais combinados: {len(final_df)} registros")
        return final_df
    else:
        print("Nenhum dado encontrado para a semana!")
        return pd.DataFrame()

In [ ]:
def main():
    ACCOUNT = os.getenv("SEMS_ACCOUNT", "demo@goodwe.com")
    PASSWORD = os.getenv("SEMS_PASSWORD", "GoodweSems123!@#")
    INVERTER_SN = "5010KETU229W6177"
    START_DATE = date(2025, 9, 14)
    COLUMNS = ["Pac", "Eday", "Cbattery1", "Vbattery1", "Ipv1", "Ipv2", "Vpv1", "Vpv2"]

    try:
        df = fetch_weekly_sems_data(
            account=ACCOUNT,
            password=PASSWORD,
            inverter_sn=INVERTER_SN,
            start_date=START_DATE,
            columns=COLUMNS,
            login_region="us",
            data_region="eu"
        )

        if not df.empty:
            print(f"\n=== DADOS DA SEMANA ===")
            print(f"Período: {df['time'].min()} até {df['time'].max()}")
            print(f"Total de registros: {len(df)}")
            print(f"Colunas: {list(df.columns)}")

            filename = f"sems_weekly_{INVERTER_SN}_{START_DATE.strftime('%Y%m%d')}_week.csv"
            df.to_csv(filename, index=False)
            print(f"\nDados salvos em: {filename}")

            df['date'] = df['time'].dt.date
            daily_stats = df.groupby('date').agg({
                'Pac': ['mean', 'max', 'sum'],
                'Eday': 'max'
            }).round(2)

            print("\n=== ESTATÍSTICAS DIÁRIAS ===")
            print(daily_stats)

        else:
            print("Nenhum dado foi encontrado!")

    except Exception as e:
        print(f"Erro: {e}")

if __name__ == "__main__":
    main()

Buscando dados da semana começando em 2025-09-14
Login realizado com sucesso!

Dia 1/7: 2025-09-14
  Pac: 288 registros
  Eday: 288 registros
  Cbattery1: 288 registros
  Vbattery1: 288 registros
  Ipv1: 288 registros
  Ipv2: 288 registros
  Vpv1: 288 registros
  Vpv2: 288 registros

Dia 2/7: 2025-09-15
  Pac: 288 registros
  Eday: 288 registros
  Cbattery1: 288 registros
  Vbattery1: 288 registros
  Ipv1: 288 registros
  Ipv2: 288 registros
  Vpv1: 288 registros
  Vpv2: 288 registros

Dia 3/7: 2025-09-16
  Pac: 288 registros
  Eday: 288 registros
  Cbattery1: 288 registros
  Vbattery1: 288 registros
  Ipv1: 288 registros
  Ipv2: 288 registros
  Vpv1: 288 registros
  Vpv2: 288 registros

Dia 4/7: 2025-09-17
  Pac: 288 registros
  Eday: 288 registros
  Cbattery1: 288 registros
  Vbattery1: 288 registros
  Ipv1: 288 registros
  Ipv2: 288 registros
  Vpv1: 288 registros
  Vpv2: 288 registros

Dia 5/7: 2025-09-18
  Pac: 288 registros
  Eday: 288 registros
  Cbattery1: 288 registros
  Vbatt

In [ ]:
df_energy = pd.read_csv('/content/sems_weekly_5010KETU229W6177_20250914_week.csv')
df_energy.head()

,time,Pac,Eday,Cbattery1,Vbattery1,Ipv1,Ipv2,Vpv1,Vpv2
0,2025-09-14 00:00:00,0.0,0.0,86.0,299.1,0.0,0.0,0.0,0.0
1,2025-09-14 00:05:00,0.0,0.1,85.0,298.8,0.0,0.0,0.0,0.0
2,2025-09-14 00:10:00,0.0,0.1,85.0,298.8,0.0,0.0,0.0,0.0
3,2025-09-14 00:15:00,0.0,0.1,84.0,299.1,0.0,0.0,0.0,0.0
4,2025-09-14 00:20:00,0.0,0.1,84.0,299.3,0.0,0.0,0.0,0.0


Pac = Potência Ativa de Saída (AC) - A potência real que o inversor está entregando para a rede em Watts (W)

Eday = Energia Gerada no Dia - Total de energia produzida no dia em Watt-hora (Wh) ou kWh

Cbattery1 = Corrente da Bateria- Fluxo de corrente da bateria (positivo = carregando, negativo = descarregando) em Amperes (A)

Vbattery1 = Tensão da Bateria 1 - Voltagem da bateria em Volts (V)

Ipv1 e Ipv2	= Tensão 1 - Voltagem da bateria em Volts (V), Cada "Ipv" representa um conjunto de painéis conectados em série

Vpv1 e Vpv2	= Tensão de cada string fotovoltaica em Volts (V)

In [ ]:
df_consumption = pd.read_csv('/content/household_power_consumption17_12_06-23_12_06.csv', parse_dates=['DateTime'])
df_consumption.drop('Date', axis=1, inplace=True)
df_consumption.drop('DateTime', axis=1, inplace=True)
df_consumption['time'] = df_energy['time']
df_consumption.drop('Time', axis=1, inplace=True)
df_consumption['Sub1_on'] = 0
df_consumption['Sub2_on'] = 0
df_consumption['Sub3_on'] = 0
df_consumption.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,time,Sub1_on,Sub2_on,Sub3_on
0,1.044,0.152,242.73,4.4,0.0,2.0,0.0,2025-09-14 00:00:00,0,0,0
1,2.848,0.198,240.59,11.8,0.0,1.0,0.0,2025-09-14 00:05:00,0,0,0
2,2.608,0.298,239.45,10.8,0.0,1.0,0.0,2025-09-14 00:10:00,0,0,0
3,1.658,0.166,238.48,8.0,0.0,1.0,0.0,2025-09-14 00:15:00,0,0,0
4,2.338,0.000,238.37,9.8,0.0,1.0,0.0,2025-09-14 00:20:00,0,0,0


Global_active_power =  potência ativa média por minuto das residências (em quilowatts)

Global_reactive_power = potência reativa média por minuto das residências (em quilowatts)

Voltage = tensão média por minuto (em volts)

Global_intensity = intensidade de corrente média por minuto global doméstica (em ampères)

Sub_metering_1 = Corresponde à cozinha, contendo principalmente uma máquina de lavar louça, um forno e um micro-ondas. (em watts-hora de energia ativa)

Sub_metering_2 = Corresponde à lavanderia, contendo uma máquina de lavar, uma secadora, uma geladeira e uma luz.(em watts-hora de energia ativa)

Sub_metering_3 = Corresponde a um aquecedor elétrico de água e um ar condicionado. (em watts-hora de energia ativa)

In [ ]:
df_energy['Pdc'] = (df_energy['Ipv1'] * df_energy['Vpv1']) + (df_energy['Ipv2'] * df_energy['Vpv2'])
df_energy['time'] = pd.to_datetime(df_energy['time']) # Convert to datetime objects
df_energy['time_diff_h'] = df_energy['time'].diff().dt.total_seconds().div(3600).fillna(0)
df_energy['Ecalc_Wh'] = df_energy['Pdc'] * df_energy['time_diff_h']  # Wh no intervalo

df_energy["week_day"] = df_energy["time"].dt.day_name()

Pdc = Potência elétrica no lado CC (corrente contínua) dos painéis solares.

time_diff_h = tempo entre um dado e outro

Ecalc_Wh = quanta energia foi gerada no intervalo a cada 5 minutos (Wh).

In [ ]:
df_energy.head()

,time,Pac,Eday,Cbattery1,Vbattery1,Ipv1,Ipv2,Vpv1,Vpv2,Pdc,time_diff_h,Ecalc_Wh,week_day
0,2025-09-14 00:00:00,0.0,0.0,86.0,299.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,Sunday
1,2025-09-14 00:05:00,0.0,0.1,85.0,298.8,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,Sunday
2,2025-09-14 00:10:00,0.0,0.1,85.0,298.8,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,Sunday
3,2025-09-14 00:15:00,0.0,0.1,84.0,299.1,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,Sunday
4,2025-09-14 00:20:00,0.0,0.1,84.0,299.3,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,Sunday


In [ ]:
def combinar_bancos_dados(arquivo1, arquivo2):

    # Ensure 'time' column is datetime type in both dataframes
    arquivo1['time'] = pd.to_datetime(arquivo1['time'])
    arquivo2['time'] = pd.to_datetime(arquivo2['time'])

    # Selecionar colunas específicas (ajuste conforme suas necessidades)
    colunas_selecionadas_df1 = ['time','week_day', 'Pac', 'Eday', 'Cbattery1','Vbattery1','Pdc','Ecalc_Wh']
    colunas_selecionadas_df2 = ['time', 'Global_active_power', 'Global_reactive_power', 'Voltage','Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3', 'Sub1_on','Sub2_on','Sub3_on']

    # Filtrar colunas
    df1_filtrado = arquivo1[colunas_selecionadas_df1]
    df2_filtrado = arquivo2[colunas_selecionadas_df2]

    # Combinar os dados usando a coluna 'time'
    df_combinado = pd.merge(df1_filtrado, df2_filtrado, on='time', how='inner')

    # Salvar resultado
    filename = "dados_combinados.csv"
    df_combinado.to_csv(filename, index=False)
    print(f"Arquivo '{filename}' criado com sucesso!")
    print(f"Total de registros: {len(df_combinado)}")

    display(df_combinado.head())

# Usar a função
combinar_bancos_dados(df_energy, df_consumption)

Arquivo 'dados_combinados.csv' criado com sucesso!
Total de registros: 2016


,time,week_day,Pac,Eday,Cbattery1,Vbattery1,Pdc,Ecalc_Wh,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub1_on,Sub2_on,Sub3_on
0,2025-09-14 00:00:00,Sunday,0.0,0.0,86.0,299.1,0.0,0.0,1.044,0.152,242.73,4.4,0.0,2.0,0.0,0,0,0
1,2025-09-14 00:05:00,Sunday,0.0,0.1,85.0,298.8,0.0,0.0,2.848,0.198,240.59,11.8,0.0,1.0,0.0,0,0,0
2,2025-09-14 00:10:00,Sunday,0.0,0.1,85.0,298.8,0.0,0.0,2.608,0.298,239.45,10.8,0.0,1.0,0.0,0,0,0
3,2025-09-14 00:15:00,Sunday,0.0,0.1,84.0,299.1,0.0,0.0,1.658,0.166,238.48,8.0,0.0,1.0,0.0,0,0,0
4,2025-09-14 00:20:00,Sunday,0.0,0.1,84.0,299.3,0.0,0.0,2.338,0.000,238.37,9.8,0.0,1.0,0.0,0,0,0


In [ ]:
df_combinado = pd.read_csv('/content/dados_combinados.csv')

In [ ]:
pip install influxdb3-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 13.5 MB/s eta 0:00:00


In [ ]:
import os, time
from influxdb_client_3 import InfluxDBClient3, Point
from influxdb_client import InfluxDBClient


bucket = "projetochallenge"
token = os.environ.get("INFLUXDB_TOKEN")
org = "projetoChallenge"
host = "https://us-east-1-1.aws.cloud2.influxdata.com"

client = InfluxDBClient3(host=host, token=token, org=org)

In [ ]:
from influxdb_client.client.write_api import ASYNCHRONOUS


client = InfluxDBClient3(host= host, token=token, org=org)

# Converter a coluna 'time' para datetime
df_combinado['time'] = pd.to_datetime(df_combinado['time'])

# Escrever dados
write_api = client.write_api(write_options=ASYNCHRONOUS)

try:
    write_api.write(
        bucket=bucket,
        record=df_combinado,
        data_frame_measurement_name='energy_data',
        data_frame_tag_columns=['week_day', 'Sub1_on', 'Sub2_on', 'Sub3_on']
    )
    print("Dados importados com sucesso!")

except Exception as e:
    print(f"Erro: {e}")

finally:
    client.close()

AttributeError: 'InfluxDBClient3' object has no attribute 'write_api'